In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from Funcoes_auxiliares.func_aux import *
from sklearn.impute import KNNImputer

# Knowing the dataset
At first, no cleaning will be applied to the dataset, later we will apply all cleaning and make a new EDA

In [ ]:
#Import dataset
df = pd.read_csv('base_suja/base_unificada_suja.csv')
variaveis = [
    'data_evento', 'ano_evento', 'TIPOBITO', 'evento_MUNNOMEX',
    'res_MUNNOMEX', 'evento_CAPITAL', 'res_CAPITAL', 'evento_REGIAO',
    'res_REGIAO', 'evento_SIGLA_UF', 'res_SIGLA_UF', 'IDADEMAE',
    'idademae_faixa', 'ESCMAE2010', 'escolaridade_mae', 'OBITOGRAV',
    'GRAVIDEZ', 'tipo_gravidez', 'SEMAGESTAC', 'idade_gestacao_faixa',
    'SEXO', 'def_sexo', 'PESO', 'peso_faixa', 'OBITOPARTO',
    'def_obito_parto', 'CAUSABAS', 'causabas_capitulo',
    'causabas_categoria', 'causabas_grupo', 'causabas_subcategoria',
    'FLAG_BASE', 'sum_CENTROBS', 'sum_QTINST34', 'sum_QTINST35',
    'sum_QTINST36', 'sum_QTINST37', 'sum_QTLEIT34', 'sum_QTLEIT38',
    'sum_QTLEIT39', 'sum_QTLEIT40', 'sum_CENTRNEO','TP_UNID_5', 'TP_UNID_7',
    'TP_UNID_15', 'TP_UNID_36', 'TP_UNID_61'
]
df = df[variaveis]

<ipython-input-2-5c667751b380>:2: DtypeWarning: Columns (25,26,27,28,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('base_suja/base_unificada_suja.csv')


In [ ]:
# Configuration to show more rows and columns
pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

In [ ]:
url_dicionario = 'https://docs.google.com/spreadsheets/d/1QFy_F2o81ULglNx8knNqg6oI7v3RARaUAaLLwOQr7K4/edit?usp=sharing'
f"""The dataset has {len(df)} lines and {len(df.columns)} columns. More details can be consulted in the dictionary {url_dicionario}. Below the type of variables and the 'face' from the dataset"""

In [ ]:
df.info()

In [ ]:
df.head()

#  Checking data quality

### Duplicates

In [ ]:
# Duplicate data 
duplicados = df.duplicated()
soma = duplicados.sum()
f"""The dataset has {soma} duplicate rows, which represents {round((soma/len(df)) * 100, 2)} %"""

In [ ]:
# Frequency of duplicates by FLAG_BASE, res_SIGLA_UF and ano_evento
df_duplicados = df[duplicados]
df_duplicados.value_counts(['FLAG_BASE', 'ano_evento', 'res_SIGLA_UF'])

# There is no concentration of missing items that indicates a structural problem with filling in the data, whether by year or 
# UF of residence

### Missing

In [ ]:
# Counting missing values
df_sim_dofet = df[df['FLAG_BASE']=='SIM_DOFET']
missing_count_sim_dofet = df_sim_dofet.isnull().sum()  # counts the null values in each column
missing_percent_sim_dofet = round((missing_count_sim_dofet / len(df_sim_dofet)) * 100,2)  # calculates the percentage of null values
missing_data_sim_dofet = pd.DataFrame({'Missing Count': missing_count_sim_dofet, 'Missing Percentage': missing_percent_sim_dofet})
missing_data_sim_dofet.index.name = 'Variable'  # sets the index name to 'Variable'
missing_data_sim_dofet.reset_index(inplace=True)  # resets the index to make 'Variable' a column
missing_data_sim_dofet['BASE'] = 'SIM_DOFET'

df_sinasc = df[df['FLAG_BASE']=='SINASC']
missing_count_sinasc = df_sinasc.isnull().sum()  # counts the null values in each column
missing_percent_sinasc = round((missing_count_sinasc / len(df_sinasc)) * 100,2)  # calculates the percentage of null values
missing_data_sinasc = pd.DataFrame({'Missing Count': missing_count_sinasc, 'Missing Percentage': missing_percent_sinasc})
missing_data_sinasc.index.name = 'Variable'  # sets the index name to 'Variable'
missing_data_sinasc.reset_index(inplace=True)  # resets the index to make 'Variable' a column
missing_data_sinasc['BASE'] = 'SINASC'

# appending
missing_data = pd.concat([missing_data_sim_dofet, missing_data_sinasc])

# Sorting the DataFrame by the highest missing frequencies
missing_data_sorted = missing_data.sort_values(by=['BASE', 'Missing Count'], ascending=False)

missing_data_sorted[missing_data_sorted['Missing Percentage'] > 0]

# Some variables are not filled in the sinasc dataset. For EDA it will be used in SIM_DOFET, but in the model it will not be removed
# 'OBITOGRAV' no padding at the base
# Missing points for other variables will be removed, keeping the variables in the study

In [ ]:
del missing_data_sorted, missing_data, df_sim_dofet, df_sinasc

In [ ]:
# Checking if there is any pattern in the missing data looking at year and 'res_SIGLA_UF'
lista = ['ESCMAE2010', 'SEMAGESTAC', 'PESO', 'IDADEMAE']
for i in lista:
    print(f"""Variável {i}\n\n{df.loc[df[i].isnull(), ['FLAG_BASE', 'ano_evento', 'res_SIGLA_UF']].value_counts()}""")

# Some UF's have very high values, but compared to the total UF it is less than 10%
# len(df.loc[(df['res_SIGLA_UF']=='CE') & (df['ano_evento']==2021) & (df['FLAG_BASE']=='SINASC')])

### Ignorado

In [ ]:
df_ignorado = df.loc[(df['ESCMAE2010']==9) | (df['GRAVIDEZ']==9) |
                     (df['idade_gestacao_faixa']=='Ignorado') | (df['SEXO']==0)]
df_ignorado['FLAG_BASE'].value_counts()

### Categorical variables

In [ ]:
lista_cat = [
'ano_evento'
, 'TIPOBITO'       
, 'evento_MUNNOMEX'
, 'res_MUNNOMEX'
, 'evento_CAPITAL'
, 'res_CAPITAL'
, 'evento_REGIAO'
, 'res_REGIAO'
, 'evento_SIGLA_UF'
, 'res_SIGLA_UF'
, 'idademae_faixa'
, 'ESCMAE2010'
, 'escolaridade_mae'
, 'OBITOGRAV'
, 'GRAVIDEZ'
, 'tipo_gravidez'
, 'idade_gestacao_faixa'
, 'SEXO'
, 'def_sexo'
, 'peso_faixa'
, 'OBITOPARTO'
, 'def_obito_parto'
, 'CAUSABAS'
, 'causabas_capitulo'
, 'causabas_categoria'
, 'causabas_grupo'
, 'causabas_subcategoria'
, 'FLAG_BASE'
            ]
for col in lista_cat:
    print(f'\nPercentual de valores únicos para {col}:')
    print(round((df[col].value_counts()/len(df)) * 100, 2))

### Numerical variables

In [ ]:
# Basic Statistics for Numeric Variables
lista_numerica = [
'IDADEMAE'
, 'SEMAGESTAC'
, 'PESO'
, 'sum_CENTROBS'
, 'sum_QTINST34'
, 'sum_QTINST35'
, 'sum_QTINST36'
, 'sum_QTINST37'
, 'sum_QTLEIT34'
, 'sum_QTLEIT38'
, 'sum_QTLEIT39'
, 'sum_QTLEIT40'
, 'sum_CENTRNEO'
, 'TP_UNID_5'
, 'TP_UNID_7'
, 'TP_UNID_15'
, 'TP_UNID_36'
, 'TP_UNID_61'
]
estatisticas_numericas = df[lista_numerica].describe()
estatisticas_numericas

### Outlier

In [ ]:
def detectar_outliers(col):
    q1 = col.quantile(0.25)
    q3 = col.quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    outliers = col[(col < lower_bound) | (col > upper_bound)]
    return outliers

outliers_numericos = df[lista_numerica].apply(detectar_outliers)

for i in lista_numerica:
    print(outliers_numericos.loc[~outliers_numericos[i].isna(), [i]].describe())

In [ ]:
for i in lista_numerica:
    x1 = df.loc[df['ano_evento'] == 2019, i]
    x2 = df.loc[df['ano_evento'] == 2020, i]
    x3 = df.loc[df['ano_evento'] == 2021, i]
    x4 = df.loc[df['ano_evento'] == 2022, i]

    # Normalize
    kwargs = dict(alpha=0.5, bins=100, density=True, stacked=True)

    # Plot
    plt.figure()  # Creates a new figure for each iteration
    plt.hist(x1, **kwargs, color='g', label='2019')
    plt.hist(x2, **kwargs, color='b', label='2020')
    plt.hist(x3, **kwargs, color='r', label='2021')
    plt.hist(x4, **kwargs, color='y', label='2022')
    plt.gca().set(title=f'Histograma {i}', ylabel='Probability')
    # plt.xlim(50, 75)
    plt.legend()

    # Show or save the figure here if needed
    # plt.savefig(f'histograma_{i}.png')

    plt.show()  # Show the graph

### Cause of death

In [ ]:
contingency_table = pd.crosstab(index=df['CAUSABAS'], columns=df['ano_evento'])#, normalize='index')
contingency_table = contingency_table.reset_index()
contingency_table.loc[contingency_table['CAUSABAS'].isin(['P200', 'P95'])]

### Births by municipality per year

In [ ]:
df_sinasc = df.loc[df['FLAG_BASE']=='SINASC']
pd.crosstab(index=df_sinasc['evento_MUNNOMEX'], columns=df_sinasc['ano_evento'])

# Cleaning and input data missing the dataset

### Input base SIM_DOFET

In [ ]:
var_input = ['ano_evento','res_REGIAO', 'evento_REGIAO', 'IDADEMAE', 'escolaridade_mae', 'tipo_gravidez'
             , 'SEMAGESTAC', 'SEXO', 'PESO']
df_sim = df.loc[df['FLAG_BASE']=='SIM_DOFET']
df_sim_dofet = df_sim[var_input]
df_sim_dofet = df_sim_dofet.reset_index(drop=True)

In [ ]:
# Acrescentando missings em Ignorados
# Substituir 'Ignorado' por NaN nas variáveis categóricas especificadas
df_sim_dofet[
    [
         'escolaridade_mae'
         , 'tipo_gravidez'
    ]
] = df_sim_dofet[
    [
         'escolaridade_mae'
        , 'tipo_gravidez'
    ]
].replace('Ignorado', np.nan)

In [ ]:
df_sim_dofet.loc[
    (df_sim_dofet['IDADEMAE'] < 10) | (df_sim_dofet['IDADEMAE'] > 50) , 'IDADEMAE'
] = np.nan

In [ ]:
df_sim_dofet.loc[
    (df_sim_dofet['SEMAGESTAC'] > 42) , 'SEMAGESTAC'
] = np.nan

In [ ]:
# Inserindo NaN em PESO
# Calcular os limites inferiores e superiores para PESO
std_peso = np.std(df['PESO'])
media_peso = np.mean(df['PESO'])
lim_inf = media_peso - (6 * std_peso)
lim_sup = media_peso + (6 * std_peso)

# Substituir valores fora do intervalo especificado por NaN na variável 'PESO'
df_sim_dofet.loc[
    (df_sim_dofet['PESO'] <= 0) | (df_sim_dofet['PESO'] < lim_inf) | (df_sim_dofet['PESO'] > lim_sup), 'PESO'
] = np.nan


In [ ]:
df_sim_dofet.loc[df_sim_dofet['SEXO'] == 0, 'SEXO'] = np.nan

In [ ]:
# Contando os Missings em cada coluna
df_sim_dofet.isnull().sum()

In [ ]:
# Converter variáveis categóricas para numéricas temporariamente
df_sim_dofet_encoded = pd.get_dummies(
    df_sim_dofet, columns=[
        'ano_evento'
        , 'res_REGIAO'
        , 'evento_REGIAO'
        , 'escolaridade_mae'
        , 'tipo_gravidez'
        , 'SEXO'
    ]
    , drop_first=False
)

# Imputação usando KNN
knn_imputer = KNNImputer(n_neighbors=5)
df_imputed_array = knn_imputer.fit_transform(df_sim_dofet_encoded)
df_imputed = pd.DataFrame(df_imputed_array, columns=df_sim_dofet_encoded.columns)

# Reverter a codificação dummy para categorias originais mantendo os nomes originais
for col in ['ano_evento', 'res_REGIAO', 'evento_REGIAO', 'escolaridade_mae', 'tipo_gravidez', 'SEXO']:
    dummy_cols = [c for c in df_imputed.columns if c.startswith(col + '_')]
    df_imputed[col] = df_imputed[dummy_cols].idxmax(axis=1).apply(lambda x: x[len(col) + 1:])
    df_imputed.drop(dummy_cols, axis=1, inplace=True)

In [ ]:
df_imputed.isnull().sum()

In [ ]:
df_sim = df_sim.reset_index(drop=True)
df_imputed = df_imputed.reset_index(drop=True)
df_sim['ano_evento'] = df_imputed['ano_evento']
df_sim['res_REGIAO'] = df_imputed['res_REGIAO']
df_sim['evento_REGIAO'] = df_imputed['evento_REGIAO']
df_sim['IDADEMAE'] = df_imputed['IDADEMAE']
df_sim['escolaridade_mae'] = df_imputed['escolaridade_mae']
df_sim['tipo_gravidez'] = df_imputed['tipo_gravidez']
df_sim['SEMAGESTAC'] = df_imputed['SEMAGESTAC']
df_sim['SEXO'] = df_imputed['SEXO']
df_sim['PESO'] = df_imputed['PESO']
df_sim['SEXO'] = df_sim['SEXO'].astype(float)
df_sim['SEXO'] = df_sim['SEXO'].astype(int)

In [ ]:
# Inserir pós input
df_sim['def_sexo'] = np.where(df_sim['SEXO']==1, 'Masculino', 'Feminino')
df_sim['idade_gestacao_faixa'] = [func_categorize_idade_gest(int(round(i,0))) for i in df_sim['SEMAGESTAC']]
df_sim['idademae_faixa'] = [func_categorize_idademae(int(round(i, 0))) for i in df_sim['IDADEMAE']]
df_sim['peso_faixa'] = [func_categorize_peso(round(i,0)) for i in df_sim['PESO']]

In [ ]:
df_sim.isnull().sum()

In [ ]:
len(df)

In [ ]:
pd.unique(df_sim['FLAG_BASE'])

In [ ]:
# Apendando as bases
df = df.loc[df['FLAG_BASE']=='SINASC']
df = pd.concat([df, df_sim])

In [ ]:
len(df)

In [ ]:
pd.unique(df['FLAG_BASE'])

In [ ]:
df[df['FLAG_BASE'] == 'SIM_DOFET'].value_counts(['ano_evento'])

### Remoção

In [ ]:
# Removing columns with a high frequency of missings

#df_limpo = df_limpo.drop(columns=['OBITOGRAV', 'causabas_categoria', 'TIPOBITO', 'causabas_capitulo', 'CAUSABAS'
#                                  , 'def_obito_parto', 'OBITOPARTO', 'causabas_subcategoria', 'causabas_grupo'])

In [ ]:
# Removing duplicates SINASC
tam_inicial = len(df)
df_sinasc = df.loc[df['FLAG_BASE']=='SINASC']
df_sim =  df.loc[df['FLAG_BASE']=='SIM_DOFET']
df_limpo = df_sinasc.drop_duplicates()
tam_depois=len(df_limpo)
print(f'Remove duplicates {tam_inicial - tam_depois} rows')

In [ ]:
# Removendo os missing SINASC

#lista_missing_sim = [
#   'sum_CENTROBS','sum_QTINST34','sum_QTINST35','sum_QTINST36','sum_QTINST37','TP_UNID_5'
#    ,'TP_UNID_7','TP_UNID_15','TP_UNID_36','TP_UNID_61', 'res_MUNNOMEX', 'res_CAPITAL', 'res_SIGLA_UF'
#    , 'evento_MUNNOMEX', 'evento_CAPITAL', 'evento_REGIAO', 'evento_SIGLA_UF', 'causabas_capitulo', 'causabas_categoria'
#    , 'causabas_grupo', 'causabas_subcategoria'
#]

lista_missing_sinasc = [
    'ESCMAE2010','SEMAGESTAC','PESO','IDADEMAE','sum_CENTROBS','sum_QTINST34','sum_QTINST35','sum_QTINST36','sum_QTINST37'
    ,'sum_QTLEIT34','sum_QTLEIT38','sum_QTLEIT39','sum_QTLEIT40','sum_CENTRNEO','TP_UNID_5'
    ,'TP_UNID_7','TP_UNID_15','TP_UNID_36','TP_UNID_61', 'GRAVIDEZ', 'res_MUNNOMEX', 'res_CAPITAL', 'res_SIGLA_UF'
    , 'evento_MUNNOMEX', 'evento_CAPITAL', 'evento_REGIAO', 'evento_SIGLA_UF'
]
tam_antes = len(df_limpo)
#df_limpo_sim = df_limpo_sim.dropna(subset = lista_missing_sim)
df_limpo = df_limpo.dropna(subset = lista_missing_sinasc)
df_limpo = df_limpo.reset_index(drop=True)
tam_depois=len(df_limpo)
print(f'Remove missing {tam_antes - tam_depois} rows')

In [ ]:
# Removendo missing SIM, somente nas colunas 'CNES'
lista_missing_sim = [
   'sum_CENTROBS','sum_QTINST34','sum_QTINST35','sum_QTINST36','sum_QTINST37','TP_UNID_5'
    ,'TP_UNID_7','TP_UNID_15','TP_UNID_36','TP_UNID_61'
]

tam_sim = len(df_sim)
df_sim = df_sim.dropna(subset = lista_missing_sim)
df_sim = df_sim.reset_index(drop=True)
tam_depois_sim = len(df_sim)
print(f'Remove missing {tam_sim - tam_depois_sim} rows')

In [ ]:
# Remove ignored class SINASC
tam_antes = len(df_limpo) 
df_limpo['evento_MUNNOMEX'] = df_limpo['evento_MUNNOMEX'].astype(str).fillna('IGNORADO')
df_limpo = df_limpo.loc[~df_limpo['evento_MUNNOMEX'].str.contains('IGNORADO')]
tam_depois=len(df_limpo)
print(f'Remove ignored evento_MUNNOMEX {tam_antes - tam_depois} rows')

In [ ]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[~df_limpo['res_MUNNOMEX'].str.contains('IGNORADO')]
tam_depois=len(df_limpo)
print(f'Remove ignored res_MUNNOMEX {tam_antes - tam_depois} rows')

In [ ]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['idademae_faixa']!='Ignorado']
tam_depois=len(df_limpo)
print(f'Remove ignored idademae_faixa {tam_antes - tam_depois} rows')

In [ ]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['ESCMAE2010']!=9]
tam_depois=len(df_limpo)
print(f'Remove ignored ESCMAE2010 {tam_antes - tam_depois} rows')

In [ ]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['GRAVIDEZ']!=9]
tam_depois=len(df_limpo)
print(f'Remove ignored GRAVIDEZ {tam_antes - tam_depois} rows')

In [ ]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['idade_gestacao_faixa']!='Ignorado']
tam_depois=len(df_limpo)
print(f'Remove ignored idade_gestacao_faixa {tam_antes - tam_depois} rows')

In [ ]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['SEXO']!=0]
tam_depois=len(df_limpo)
print(f'Remove ignored SEXO {tam_antes - tam_depois} rows')

In [ ]:
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['peso_faixa']!='Ignorado']
tam_depois=len(df_limpo)
print(f'Remove ignored peso_faixa {tam_antes - tam_depois} rows')

In [ ]:
# Mother's age
# Remover somente em SINASC
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[(df_limpo['IDADEMAE']>=10) & (df_limpo['IDADEMAE']<=50)]
tam_depois=len(df_limpo)
print(f'Remove IDADEMAE <10 or >50 {tam_antes - tam_depois} rows')

In [ ]:
# Weight
std_peso = np.std(df['PESO'])
media_peso = np.mean(df['PESO'])
lim_inf = media_peso - (6 * std_peso)
lim_sup = media_peso + (6 * std_peso)
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['PESO']>0]
df_limpo = df_limpo.loc[(df_limpo['PESO']>=lim_inf) & (df_limpo['PESO']<=lim_sup)]
tam_depois=len(df_limpo)
print(f'Remove PESO 6 std {tam_antes - tam_depois} rows')

In [ ]:
# Weeks of pregnancy less than 22 are not considered deaths, but rather miscarriages. SIM e SINASC
df_limpo = pd.concat([df_limpo, df_sim])
tam_antes = len(df_limpo)
df_limpo = df_limpo.loc[df_limpo['idade_gestacao_faixa']!='menor_22']
tam_depois=len(df_limpo)
print(f'Remove idade_gestacao_faixa menor_22 {tam_antes - tam_depois} rows')

In [ ]:
tam_depois = len(df)
print(f'Total remove {tam_inicial - tam_depois} rows')

In [ ]:
df_limpo = df_limpo.reset_index(drop=True)

In [ ]:
df_limpo.info()

In [ ]:
df_limpo[df_limpo['FLAG_BASE'] == 'SIM_DOFET'].value_counts(['ano_evento'])

In [ ]:
df_limpo.to_csv('base_limpa/base_unificada_limpa_com_input.csv', index=False)